# Import Files

In [9]:
import moviepy.editor as mp
import speech_recognition as sr
from io import BytesIO
import numpy as np

# Load video

In [3]:
video = mp.VideoFileClip("a.mp4")

# Extract from audio to video

In [10]:
audio = video.audio
audio_chunks = audio.iter_chunks(fps=16000, chunksize=100000)

# Stack the chunks into a full audio array
audio_array = np.vstack(list(audio_chunks))


# Initialize Recognizer

In [11]:
r = sr.Recognizer() 

# Load audio

In [12]:
with sr.AudioFile(BytesIO(audio_array.tobytes())) as source:
    # Record the audio in chunks from the memory stream
    audio_data = r.record(source)

    # Convert speech to text using Google API
    text = r.recognize_google(audio_data)


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [13]:
# Print the transcription
print("\nThe resultant text from video is: \n")
print(text)



The resultant text from video is: 



NameError: name 'text' is not defined

In [21]:
with sr.AudioFile("a.wav") as source:
    data = r.record(source)

# Convert speech to text

In [22]:
text = r.recognize_google(data) 

In [23]:
print("\nThe resultant text from video is: \n") 
print(text) 


The resultant text from video is: 

good morning everyone my name is Ayush Jindal and currently I am in fourth year


In [24]:
save_text_to_pdf(text)

Text saved to transcribed_text.pdf


In [25]:
with open("a.txt", "w") as file:
        file.write(text)

In [28]:
import moviepy.editor as mp
import speech_recognition as sr
import numpy as np
from io import BytesIO
import wave

# Function to convert numpy array of audio to a byte stream in WAV format
def numpy_to_wav(audio_array, fps=16000):
    # Normalize the array and convert it to int16 for WAV format
    audio_array = (audio_array * 32767).astype(np.int16)

    # Prepare an in-memory buffer to store the audio
    wav_buffer = BytesIO()

    # Write WAV header and audio content to the buffer
    with wave.open(wav_buffer, "wb") as wav_file:
        wav_file.setnchannels(1)  # Mono audio
        wav_file.setsampwidth(2)  # Each sample is 2 bytes (int16)
        wav_file.setframerate(fps)
        wav_file.writeframes(audio_array.tobytes())

    # Move back to the start of the buffer
    wav_buffer.seek(0)

    return wav_buffer

# Function to transcribe video to text using speech recognition
def transcribe_video_to_text(video_path):
    try:
        # Load the video file
        video = mp.VideoFileClip(video_path)

        # Check if the video has an audio track
        if video.audio is None:
            print("The video does not contain an audio track.")
            return

        # Extract audio chunks and process them
        audio = video.audio
        audio_chunks = []

        # Iterate over audio chunks
        for chunk in audio.iter_chunks(fps=16000, chunksize=100000):
            audio_chunks.append(chunk)

        # Stack the chunks into a complete audio array
        if len(audio_chunks) > 0:
            audio_array = np.vstack(audio_chunks)

            # Convert the audio array to mono channel (if necessary)
            if audio_array.ndim > 1:
                audio_array = np.mean(audio_array, axis=1)  # Convert stereo to mono

            # Convert the numpy array to a WAV byte stream
            wav_audio = numpy_to_wav(audio_array, fps=16000)

            # Initialize the recognizer
            r = sr.Recognizer()

            # Use the WAV byte stream directly for transcription
            with sr.AudioFile(wav_audio) as source:
                audio_data = r.record(source)  # Record the entire audio from the source
                text = r.recognize_google(audio_data)  # Use Google Speech-to-Text

            # Output the transcribed text
            print("\nThe resultant text from the video is:\n")
            print(text)

            # Optionally, save the transcription to a text file or PDF
            with open("c.txt", "w") as file:
                file.write(text)
            print("\nText saved to transcribed_text.txt")

        else:
            print("No audio data extracted.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
video_path = "sample.mp4"  # Replace with your video file path
transcribe_video_to_text(video_path)


An error occurred: 


In [30]:
!pip install pydub

Defaulting to user installation because normal site-packages is not writeable
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [37]:
import moviepy.editor as mp
import speech_recognition as sr
from pydub import AudioSegment
from io import BytesIO

# Function to transcribe video to text
def transcribe_video_to_text(video_path, chunk_size=30):
    try:
        # Load the video file
        video = mp.VideoFileClip(video_path)

        # Check if the video has an audio track
        if video.audio is None:
            print("The video does not contain an audio track.")
            return

        # Extract audio and convert it to pydub AudioSegment
        audio_path = "temp_audio.wav"
        video.audio.write_audiofile(audio_path, codec='pcm_s16le')  # Save audio to temp file
        audio_segment = AudioSegment.from_wav(audio_path)

        # Initialize recognizer
        recognizer = sr.Recognizer()
        total_duration = len(audio_segment) / 1000.0  # Duration in seconds
        transcribed_text = []

        # Process the audio in chunks
        for start_time in range(0, int(total_duration), chunk_size):
            end_time = min(start_time + chunk_size, int(total_duration))
            print(f"Transcribing from {start_time:.2f} to {end_time:.2f} seconds...")

            # Extract the audio segment for transcription
            chunk = audio_segment[start_time * 1000:end_time * 1000]  # Convert to milliseconds

            # Convert the chunk to a byte stream in WAV format
            wav_buffer = BytesIO()
            chunk.export(wav_buffer, format="wav")
            wav_buffer.seek(0)

            # Transcribe the audio chunk
            with sr.AudioFile(wav_buffer) as source:
                audio_data = recognizer.record(source)
                try:
                    text = recognizer.recognize_google(audio_data)
                    transcribed_text.append(text)
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand the audio.")
                except sr.RequestError as e:
                    print(f"Could not request results from Google Speech Recognition service; {e}")

        # Combine the transcribed text
        final_text = " ".join(transcribed_text)

        # Output the transcribed text
        print("\nThe resultant text from the video is:\n")
        print(final_text)

        # Save the transcription to a text file
        with open("transcribed_text.txt", "w") as file:
            file.write(final_text)
        print("\nText saved to transcribed_text.txt")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
video_path = "sample.mp4"  # Replace with your video file path
transcribe_video_to_text(video_path, chunk_size=30)


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
Transcribing from 0.00 to 4.00 seconds...

The resultant text from the video is:

hello my name is Ayush singer

Text saved to transcribed_text.txt


In [35]:
print(audio_array.shape)

(108000, 2)


In [38]:
import moviepy.editor as mp
import speech_recognition as sr
from pydub import AudioSegment
from io import BytesIO
import numpy as np

# Function to transcribe video to text
def transcribe_video_to_text(video_path, chunk_size=30):
    try:
        # Load the video file
        video = mp.VideoFileClip(video_path)

        # Check if the video has an audio track
        if video.audio is None:
            print("The video does not contain an audio track.")
            return

        # Extract audio directly as numpy array
        audio_array = video.audio.to_soundarray(fps=16000)

        # Handle stereo to mono conversion if needed
        if audio_array.ndim > 1:
            audio_array = np.mean(audio_array, axis=1)  # Convert to mono

        # Convert numpy array to bytes
        audio_data = audio_array.tobytes()

        # Create an AudioSegment from the byte data
        audio_segment = AudioSegment(
            audio_data,
            frame_rate=16000,
            sample_width=2,  # 2 bytes for int16
            channels=1  # Mono
        )

        # Initialize recognizer
        recognizer = sr.Recognizer()
        total_duration = len(audio_segment) / 1000.0  # Duration in seconds
        transcribed_text = []

        # Process the audio in chunks
        for start_time in range(0, int(total_duration), chunk_size):
            end_time = min(start_time + chunk_size, int(total_duration))
            print(f"Transcribing from {start_time:.2f} to {end_time:.2f} seconds...")

            # Extract the audio segment for transcription
            chunk = audio_segment[start_time * 1000:end_time * 1000]  # Convert to milliseconds

            # Convert the chunk to a byte stream in WAV format
            wav_buffer = BytesIO()
            chunk.export(wav_buffer, format="wav")
            wav_buffer.seek(0)

            # Transcribe the audio chunk
            with sr.AudioFile(wav_buffer) as source:
                audio_data = recognizer.record(source)
                try:
                    text = recognizer.recognize_google(audio_data)
                    transcribed_text.append(text)
                except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand the audio.")
                except sr.RequestError as e:
                    print(f"Could not request results from Google Speech Recognition service; {e}")

        # Combine the transcribed text
        final_text = " ".join(transcribed_text)

        # Output the transcribed text
        print("\nThe resultant text from the video is:\n")
        print(final_text)

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
video_path = "a.mp4"  # Replace with your video file path
transcribe_video_to_text(video_path, chunk_size=30)

An error occurred: arrays to stack must be passed as a "sequence" type such as list or tuple.
